In [5]:
from enum import StrEnum
from typing import Literal

from pydantic import BaseModel, Field

from ayy.dialog import Dialog, ModelName, exchange

In [6]:
class MemoryType(StrEnum):
    SEMANTIC = "semantic"
    EPISODIC = "episodic"


class Memory(BaseModel):
    name: str
    content: str
    confidence: float = Field(default=1.0, ge=0.0, le=1.0)


class SemanticCategory(StrEnum):
    IDENTITY = "identity"
    PREFERENCES = "preferences"
    RELATIONSHIPS = "relationships"
    SKILLS = "skills"
    BELIEFS = "beliefs"
    BACKGROUND = "background"
    HEALTH = "health"
    LOCATION = "location"
    SCHEDULE = "schedule"
    GOALS = "goals"
    OTHER = "other"


class SemanticMemory(Memory):
    """Facts and knowledge that persist across conversations"""

    type: Literal[MemoryType.SEMANTIC] = MemoryType.SEMANTIC
    category: SemanticCategory


class EpisodicMemory(Memory):
    """Experiences and events tied to specific contexts"""

    type: Literal[MemoryType.EPISODIC] = MemoryType.EPISODIC
    context: str = Field(description="Context this memory is relevant to")


class Summary(BaseModel):
    bullet_points: list[str]
    semantic_memories: list[SemanticMemory] = Field(default_factory=list)
    episodic_memories: list[EpisodicMemory] = Field(default_factory=list)

In [7]:
SUMMARIZE_MESSAGES = """
You are a skilled conversation summarizer. Your task is to analyze a conversation and create a concise, high-level summary that significantly reduces the message volume while preserving essential information.

The main purpose of summarization is to condense lengthy conversations into a brief, digestible format. Your summary should be much shorter than the original conversation - aim to capture key points and outcomes rather than listing every exchange. The conversation might already have a summary message at the start with a <summary_of_our_previous_conversation(s)> tag, that means that it was summarized before and has once again gotten too long. Make sure to incorporate that existing summary message into your summary.

Create a structured summary that includes:

1. Key bullet points that capture the essential flow and important moments of the conversation:
   - Focus on outcomes and decisions rather than back-and-forth exchanges
   - Combine related exchanges into single points
   - Skip pleasantries and redundant information
   - Highlight major state changes or decisions

2. Semantic memories that should be remembered long-term, such as:
   - Permanent facts about participants
   - Personal information
   - Long-term preferences and traits
   - Important rules or instructions
   - User feedback for improvement
   - Any other unchanging, crucial information

3. Episodic memories relevant to ongoing tasks:
   - Current context and parameters
   - Intermediate steps or results
   - Short-term preferences or needs
   - Time-sensitive details
   - Recent interactions or decisions

Focus on extracting meaningful insights while distinguishing between permanent and temporary information. Be concise but comprehensive.

Format your response as a structured Summary object with bullet_points, semantic_memories, and episodic_memories fields.
"""

In [8]:
SUMMARIZER_DIALOG = Dialog(
    model_name=ModelName.GEMINI_FLASH,
    system=SUMMARIZE_MESSAGES + f"\nAvailable categories for semantic memories: {SemanticCategory._member_names_}",
    messages=[
        *exchange(user="My name is Hamza", assistant="Hey Hamza, how can I help you today?"),
        *exchange(
            user="I'm planning a trip to visit my family in New York. How's the weather there?",
            assistant="Let me check the weather for you. It looks like it's currently sunny in New York with temperatures around 75°F (24°C). The forecast shows similar weather for the next few days.",
        ),
        *exchange(
            user="I love sunny days! That's perfect for my trip.",
            assistant="That's great that the weather aligns with your preferences! Would you like me to help you book a flight to New York?",
        ),
        *exchange(
            user="Yes please, I'd prefer a morning flight if possible",
            assistant="I'll look for morning flights. Do you have any specific dates in mind for your travel?",
        ),
        *exchange(
            user="Next weekend would be ideal",
            assistant="I found several morning flights for next weekend. There's a direct flight on Saturday at 8:30 AM, arriving at 11:45 AM. Would that work for you?",
        ),
        *exchange(
            user="That sounds perfect",
            assistant="Great! I've booked the flight for you. Here are the details:\n- Flight AA1234\n- Saturday, 8:30 AM departure\n- Direct flight to JFK\n- Confirmation code: XYZ789",
        ),
        *exchange(
            user="Thanks! Could you remind me what time I arrive?",
            assistant="Your flight arrives at JFK at 11:45 AM. Would you like me to help arrange transportation from the airport?",
        ),
        *exchange(
            user="No thanks, my family will pick me up",
            assistant="Perfect! Let me know if you need anything else regarding your trip.",
        ),
        *exchange(
            user="Summarize the conversation so far.",
            assistant="""Summary: {
                "bullet_points": [
                    "Trip planning initiated to visit family in New York",
                    "Flight booked: Saturday morning direct flight to JFK",
                    "Airport pickup arranged with family"
                ],
                "semantic_memories": [
                    {
                        "name": "personal_info",
                        "content": "User's name is Hamza",
                        "category": "identity",
                        "confidence": 1.0,
                    },
                    {
                        "name": "family_location",
                        "content": "Has family in New York",
                        "category": "relationships",
                        "confidence": 1.0,
                    },
                    {
                        "name": "travel_preferences",
                        "content": "Prefers sunny weather and morning flights",
                        "category": "preferences",
                        "confidence": 0.8,
                    }
                ],
                "episodic_memories": [
                    {
                        "name": "flight_booking",
                        "content": {
                            "flight_number": "AA1234",
                            "departure": "8:30 AM Saturday",
                            "arrival": "11:45 AM",
                            "confirmation": "XYZ789"
                        },
                        "context": "current_trip",
                        "confidence": 1.0,
                    },
                    {
                        "name": "airport_pickup",
                        "content": "Family will provide airport pickup",
                        "context": "current_trip",
                        "confidence": 1.0,
                    }
                ]
            }""",
        ),
    ],
    name="summarizer_dialog",
)